In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer

In [80]:
import sys
print(sys.executable)
train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
test = pd.read_csv('test_with_solutions.csv')

C:\Users\lenovo\Anaconda3\python.exe


In [81]:
test.head()


,Insult,Date,Comment,Usage
0,0,NaN,"""THE DRUDGE REPORT\\n\\n\\n\\nYou won't see th...",PublicTest
1,0,20120618222256Z,"""@ian21\xa0""Roger Clemens is the fucking man, ...",PublicTest
2,1,20120618213617Z,"""Agree with Alan you are an extremest idiot. ...",PublicTest
3,0,NaN,"""Really?\\n\\nI see Marc Lamont Hill on variou...",PrivateTest
4,0,20120620003825Z,"""Really suck isn't the word, when many of our ...",PrivateTest


In [82]:
#clean data
def remove_num(cmt):
     cmt = re.sub("\S*\d\S*", "", cmt).strip()
     for i in range(len(cmt)):
        if cmt[i]=='_':
            cmt = cmt[:i]+" "+cmt[i+1:]
     return cmt
train['Comment'] = train['Comment'].apply(remove_num)
test['Comment'] = test['Comment'].apply(remove_num)

In [83]:
train.head()
test.head()

,Insult,Date,Comment,Usage
0,0,NaN,"""THE DRUDGE REPORT\\n\\n\\n\\nYou won't see th...",PublicTest
1,0,20120618222256Z,"Clemens is the fucking man, and never did any ...",PublicTest
2,1,20120618213617Z,"""Agree with Alan you are an extremest idiot. ...",PublicTest
3,0,NaN,"""Really?\\n\\nI see Marc Lamont Hill on variou...",PrivateTest
4,0,20120620003825Z,"""Really suck isn't the word, when many of our ...",PrivateTest


In [84]:
def _1len(cmt):
    cmt = re.sub("\S*\?\S*", "", cmt).strip()
    cmt = re.sub("\S\\\S*","",cmt).strip()
    cmt = cmt.lower()
    return cmt
train['Comment'] = train['Comment'].apply(_1len)
test['Comment'] = test['Comment'].apply(_1len)

In [85]:
train.head(15)
test.head()

,Insult,Date,Comment,Usage
0,0,NaN,"""the drudge repor\\\\nyou won't see this story...",PublicTest
1,0,20120618222256Z,"clemens is the fucking man, and never did any ...",PublicTest
2,1,20120618213617Z,"""agree with alan you are an extremest idiot. ...",PublicTest
3,0,NaN,see marc lamont hill on various programs quite...,PrivateTest
4,0,20120620003825Z,"""really suck isn't the word, when many of our ...",PrivateTest


In [86]:
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [87]:
import string
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [88]:

test_label=test['Insult']
test.drop(['Insult'],axis=1,inplace=True)
alldata = pd.concat([train, test]).reset_index(drop=True)
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer(analyzer='word', ngram_range = (1,1), min_df=2)
vec = countvec.fit_transform(alldata['Comment'])
# vec2= countvec.fit_transform(test['Comment'],'max_features=400')


In [89]:
print(vec)

  (0, 1714)	1
  (0, 8242)	1
  (0, 2868)	1
  (0, 8238)	1
  (1, 5113)	1
  (1, 253)	1
  (1, 322)	1
  (1, 344)	1
  (1, 7389)	1
  (1, 6503)	1
  (1, 3744)	1
  (1, 7733)	1
  (1, 2100)	1
  (1, 5968)	1
  (1, 8242)	1
  (1, 8238)	1
  (2, 4307)	1
  (2, 6868)	1
  (2, 3541)	2
  (2, 4163)	1
  (2, 1721)	1
  (2, 390)	1
  (2, 6042)	1
  (2, 221)	1
  (2, 884)	1
  :	:
  (6593, 8170)	1
  (6593, 8042)	1
  (6593, 4326)	2
  (6593, 7502)	6
  (6593, 3541)	4
  (6593, 4163)	1
  (6593, 390)	2
  (6593, 3930)	1
  (6593, 6800)	1
  (6593, 5111)	1
  (6593, 3726)	2
  (6593, 7391)	7
  (6593, 5949)	1
  (6593, 136)	2
  (6593, 603)	2
  (6593, 8090)	1
  (6593, 632)	1
  (6593, 8190)	1
  (6593, 3193)	1
  (6593, 5034)	1
  (6593, 253)	7
  (6593, 344)	1
  (6593, 7389)	4
  (6593, 7733)	1
  (6593, 8238)	4


In [90]:
alldata.head()

,Comment,Date,Insult,Usage
0,"""you fuck your dad.""",20120618192155Z,1.0,NaN
1,"""i really don't understand your it seems that...",20120528192215Z,0.0,NaN
2,of canadians can and has been wrong before now...,NaN,0.0,NaN
3,"""listen if you dont wanna get married to a man...",NaN,0.0,NaN
4,"xecnh cxed giang, ...",20120619094753Z,0.0,NaN


In [91]:
bow_df = pd.DataFrame(vec.todense())

# set column names
bow_df.columns = ['col'+ str(x) for x in bow_df.columns]

bow_df_train = bow_df[:len(train)]
bow_df_test = bow_df[len(train):]

In [92]:
len(bow_df_train)

3947

In [93]:
labels=train['Insult']

In [94]:
# print(labels)
bow_df_train.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col8260,col8261,col8262,col8263,col8264,col8265,col8266,col8267,col8268,col8269
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# Initialize the constructor
model = Sequential()
# model.add(Embedding(1000,32, input_length =502))
# model.add(Flatten())
# Add an input layer
model.add(Dense(30, activation='relu', input_shape=(8270, )))

# Add one hidden layer
model.add(Dense(24, activation='relu'))

# Add an output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [96]:
model.fit(bow_df_train,labels,epochs=3, batch_size=1, verbose=1)



Epoch 1/3
3947/3947 [==============================] - 30s 8ms/step - loss: 0.4482 - acc: 0.8100
Epoch 2/3
3947/3947 [==============================] - 28s 7ms/step - loss: 0.2238 - acc: 0.9118
Epoch 3/3
3947/3947 [==============================] - 28s 7ms/step - loss: 0.1156 - acc: 0.9554


In [97]:
y_pred = model.predict(bow_df_test)

# def to_labels(x):
#     if x > 0.40:
#         return 1
#     return 0
# y_pred = y_pred.apply(to_labels)
# print(y_pred)

# y_pred=y_pred-y_pred.mean()/y_pred.max()

uu=y_pred.flatten()
uu=(uu-uu.mean())/uu.max()
print(uu)
ypred=[]
for i in range(len(uu)):
    if(uu[i]>0.4):
        ypred.append(1)
    else:
        ypred.append(0)
        

[-0.25415146  0.06301993  0.74468035 ..., -0.11142907 -0.12058482
 -0.25415146]


In [98]:

print(ypred)

[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 

In [99]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_label,ypred ))

0.832640725349
